In [16]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [17]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2021-03-03 08:45:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-03-03 08:45:23 (107 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [18]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') > DESIRED_ACCURACY):
      print('\Reached Accuracy 99.9% and stopped training')
      self.model.stop_training = True

# instantiate callback object
callbacks = myCallback()

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop

In [26]:
# Model Building
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=RMSprop(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [27]:
# ImageGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

# load it from directory and the generator directly
train_generator = train_datagen.flow_from_directory(
    "/tmp/h-or-s",
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)


# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [28]:
# Model fitting
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    callbacks=[callbacks]
)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 1s 17ms/step - loss: 2.3466 - accuracy: 0.6300
Epoch 2/15
8/8 [==============================] - 0s 17ms/step - loss: 0.5515 - accuracy: 0.6857
Epoch 3/15
8/8 [==============================] - 0s 17ms/step - loss: 0.3098 - accuracy: 0.8633
Epoch 4/15
8/8 [==============================] - 0s 19ms/step - loss: 0.3065 - accuracy: 0.9008
Epoch 5/15
8/8 [==============================] - 0s 16ms/step - loss: 0.2499 - accuracy: 0.8476
Epoch 6/15
8/8 [==============================] - 0s 17ms/step - loss: 0.1146 - accuracy: 0.9667
Epoch 7/15
8/8 [==============================] - 0s 18ms/step - loss: 0.0947 - accuracy: 0.9683
Epoch 8/15
8/8 [==============================] - 0s 17ms/step - loss: 0.1270 - accuracy: 0.9745
Epoch 9/15
8/8 [==============================] - 0s 16ms/step - loss: 0.1718 - accuracy: 0.9527
Epoch 10/15
8/8 [==============================] - 0s 16ms/step - loss: 0.0717 - accuracy: 0.9684
Epoch 11/15
8/8 [============